In [10]:
import numpy as np
import sys
import psi4
from helper_PFCI import PFHamiltonianGenerator
np.set_printoptions(threshold=sys.maxsize)
psi4.core.set_output_file('output.dat', False)
import time
import json


In [11]:
file_string = "HHep_PCQED_2880_10_cc_pVQZ_states"

mol_str = """
H       
HE            1    0.776011
1 1
symmetry c1
"""

l_array = np.array([0., 0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.011, 0.012, 0.013, 0.014, 0.015, 0.016, 0.017, 0.018, 0.019, 0.02, 0.025, 0.03, 0.035, 0.04, 0.045, 0.05, 0.055, 0.06, 0.065, 0.07, 0.075, 0.08, 0.085, 0.09, 0.095, 0.1])

options_dict = { 
        "basis": "cc-pVQZ",
        "scf_type": "pk",
        "e_convergence": 1e-10,
        "d_convergence": 1e-10,
}

cavity_dict = {
        'omega_value' : 0.9637811053,
        'lambda_vector' : np.array([0, 0, 0.01]),
        'ci_level' : 'fci',
        'davidson_roots' : 10,
        'number_of_photons' : 10,
        'davidson_threshold' : 1e-9,
        'photon_number_basis' : True,
        'canonical_mos' : True,
        'coherent_state_basis' : False
}

In [12]:
# set up base dictionary - some of this will be updated with each calculation
dictionary = {
  "repo" : {

      "repository_url" : "https://github.com/mapol-chem/qed-ci",
      "branch" : "direct_ci",
      "commit" : "7f25d1c5fd458a06002e79d610aa17399f213f71"
  },

  "molecule" : {
    "molecule_name": "HHe+",
    "geometry": [mol_str],
    "symbols": [
            "H",
            "He",
    ],
    },
    "driver": "energy",
    "model" : {
        "method" : "qed-fci",
        "orbital_basis"  : options_dict["basis"],
        "photon_basis" : "photon_number_basis",
        "number_of_photon_states" : cavity_dict["number_of_photons"],
        "lambda" : [
        ],
        "omega" : cavity_dict["omega_value"]
    },

    "return_result" : [

    ],

}

def generate_file_name(dictionary):
    file_name = dictionary["molecule"]["molecule_name"] + "_lambda_"
    file_name += str(dictionary["model"]["method"]) + "_"
    file_name += str(dictionary["model"]["orbital_basis"]) + "_"
    file_name += str(dictionary["model"]["photon_basis"]) + "_"
    file_name += str(dictionary["model"]["number_of_photon_states"]) + "_"
    #file_name += "lambda_" + str(dictionary["model"]["lambda"][0])
    #file_name += "_" + str(dictionary["model"]["lambda"][1])
    #file_name += "_" + str(dictionary["model"]["lambda"][2]) + "_"
    file_name += "omega_" + str(dictionary["model"]["omega"]) + ".json"
    return file_name


In [13]:
file_name = generate_file_name(dictionary)

In [14]:
qed_fci_qz_large_lambda = np.array([
 -2.9773960939e+00,
 -2.1846350116e+00,
 -2.0247781044e+00,
 -2.0023579319e+00,
 -1.7700528337e+00,
 -1.7270874847e+00,
 -1.7270874847e+00,
 -1.7220652946e+00,
 -1.6602926834e+00,
 -1.6602926834e+00]
)





# these file names should still be good
qz_en_file = "HHep_fci_cc-pVQZ_Energies.npy"
qz_mu_file = "HHep_fci_cc-pVQZ_Dipoles.npy"


qz_en = np.load(qz_en_file)
qz_mu = np.load(qz_mu_file)


In [15]:
# setup basic arguments to create an instance of the PFHamiltonianGenerator class
t_mol_str = """
Li
H 1 1.4
symmetry c1
"""

t_options_dict = {
    "basis": "sto-3g",
    "scf_type": "pk",
    "e_convergence": 1e-10,
    "d_convergence": 1e-10,
}


t_cavity_free_dict = {
    'omega_value' : 0.0,
    'lambda_vector' : np.array([0, 0, 0.0]),
    'ci_level' : 'fci',   
    'full_diagonalization' : True,
    'number_of_photons' : 0, 
}

# create the instance of our PFHamiltonianGenerator class
qz_inst = PFHamiltonianGenerator(t_mol_str, t_options_dict, t_cavity_free_dict)


Start SCF iterations:

Canonical RHF One-electron energy = -12.6946346236296055
CQED-RHF One-electron energy      = -12.6946346236296055
Nuclear repulsion energy          = 1.1339511657214287
Dipole energy                     = 0.0000000000000000
SCF Iteration   1: Energy = -7.8605386610570473   dE = -7.86054E+00   dRMS = 1.78646E-13
SCF Iteration   2: Energy = -7.8605386610570420   dE =  5.32907E-15   dRMS = 3.97604E-14
Total time for SCF iterations: 0.000 seconds 

QED-RHF   energy: -7.86053866 hartree
Psi4  SCF energy: -7.86053866 hartree
 Completed QED-RHF in 0.11113309860229492 seconds
 Completed 1HSO Build in 4.291534423828125e-05 seconds
 Completed ERI Build in 0.0010542869567871094 seconds 
 Completed 2D build in 9.298324584960938e-05 seconds
 Completed 1G build in 1.3113021850585938e-05 seconds
 Completed the Dipole Matrix Build in 3.886222839355469e-05 seconds
 Completed determinant list in 0.0004742145538330078 seconds 
 Completed constant offset matrix in 0.000110149383544

In [ ]:
# define 95% and 100% of electronic states
N_el = 2880


# number of photonic states
N_ph = 10

# omega
omega_qz = 0.9637811053

for l in l_array:
    l_vector = np.array([0., 0, l])
    lam_list = [0, 0, l]
    qz_inst.fast_build_pcqed_pf_hamiltonian(N_el, N_ph, omega_qz, l_vector, qz_en, qz_mu, neglect_DSE=False)
    ei = np.copy(qz_inst.PCQED_pf_eigs[:6])
    dictionary["model"]["lambda"].append(lam_list)
    dictionary["return_result"].append(list(ei))

print(dictionary)

json_object = json.dumps(dictionary, indent=4)

with open(file_name, "w") as outfile:
    outfile.write(json_object)



 Build _A and _O Arrays
Built d array
Built PCQED Hamiltonian!


In [ ]:

# build PCQED Hamiltonian
qz_inst.fast_build_pcqed_pf_hamiltonian(N_el_100_pct, N_ph, omega_qz, lambda_vector, qz_en, qz_mu, neglect_DSE=False)

In [ ]:
ei = np.copy(qz_inst.PCQED_pf_eigs[:6])
print(F' {N_el_list_qz[i]}, {ei[0]:12.10e},{ei[1]:12.10e},{ei[2]:12.10e},{ei[3]:12.10e},{ei[4]:12.10e},{ei[5]:12.10e}')